In [6]:
import numpy as np
import tinyarray
import qsymm
import sympy
from sympy import Matrix
import string
from convert_func import bloch_model_from_json, bloch_model_to_json

In [3]:
sigma_x = tinyarray.array([[0, 1], [1, 0]])
sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
sigma_z = tinyarray.array([[1, 0], [0, -1]])

#create qsymm model
ham = " cos(k_x) * sigma_z + sin(k_y) * eye(2) "
H = qsymm.Model(ham, momenta=["k_x", "k_y"])
H.tosympy()

Matrix([
[0.5*e**(I*k_x) - 0.5*I*e**(I*k_y) + 0.5*I*e**(-I*k_y) + 0.5*e**(-I*k_x),                                                                        0],
[                                                                      0, -0.5*e**(I*k_x) - 0.5*I*e**(I*k_y) + 0.5*I*e**(-I*k_y) - 0.5*e**(-I*k_x)]])

In [4]:
#find symmetries
candidates = qsymm.groups.square()
discrete_symm, continuous_symm = qsymm.symmetries(H, candidates)
print(len(discrete_symm), len(continuous_symm))

abcd_sympy = list(sympy.symbols('a:z'))
abcd_list = list(string.ascii_lowercase)
del abcd_sympy[4]
del abcd_list[4]

norbs = [('A', 2)]  # A atom per unit cell, 4 orbitals each
hopping_vectors = [('A', 'A', [0, 1]), ('A', 'A', [1, 0])] 

#find all temrs compatible with 1 specific symmetry
family= qsymm.bloch_family(hopping_vectors, [discrete_symm[1]], norbs)

#add all the terms compatible with the symmetry
H = (H + sum([symbol * term for symbol, term in zip(abcd_sympy[3:], family)]))
H.tosympy(nsimplify=True)

8 1


Matrix([
[d + e**(I*k_x)*i + e**(I*k_x)/2 + e**(I*k_y)*m + I*e**(I*k_y)*q - I*e**(I*k_y)/2 + e**(-I*k_y)*m - I*e**(-I*k_y)*q + I*e**(-I*k_y)/2 + e**(-I*k_x)*i + e**(-I*k_x)/2,                          e**(I*k_x)*j + I*e**(I*k_x)*l + e**(I*k_y)*n + I*e**(I*k_y)*r + f + I*h + e**(-I*k_y)*o - I*e**(-I*k_y)*s + e**(-I*k_x)*j + I*e**(-I*k_x)*l],
[                         e**(I*k_x)*j - I*e**(I*k_x)*l + e**(I*k_y)*o + I*e**(I*k_y)*s + f - I*h + e**(-I*k_y)*n - I*e**(-I*k_y)*r + e**(-I*k_x)*j - I*e**(-I*k_x)*l, e**(I*k_x)*k - e**(I*k_x)/2 + e**(I*k_y)*p + I*e**(I*k_y)*t - I*e**(I*k_y)/2 + g + e**(-I*k_y)*p - I*e**(-I*k_y)*t + I*e**(-I*k_y)/2 + e**(-I*k_x)*k - e**(-I*k_x)/2]])

In [5]:
bloch_model_to_json(H, 'model')